In [72]:
import os
import json
import pandas as pd
import traceback

In [73]:
from langchain.chat_models import ChatOpenAI

In [74]:
from dotenv import load_dotenv

load_dotenv() 

True

In [75]:
KEY=os.getenv("OPENAI_API_KEY")

In [76]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [77]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [78]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [79]:
TEMPLATE="""
Text:{text}
you are an expert MCQ maker. Given the above text, it is your job to \ 
create a quiz of {numbers} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it aa aguide. \
Ensure to make {numbers} MCQs
### RESPONSE_JSON
{response_json}

"""

In [80]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "numbers", "subject", "tone", "response_json"],
    template = TEMPLATE
) 

In [81]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [82]:
TEMPLATE2="""
You are an expert english grammarin and writer. Given a Multiple choice quiz for {subject} students.\
You nedd to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complaxity
if the quiz is not at per with cognitive and analytical abilities of the students,\
update the quiz questions which nedds to be changed and change the tone such that it perfectly fits the students abilities 
Quiz_MCQs:
{quiz}

check from an expert English writer of the above quiz:
"""

In [83]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [84]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [85]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "numbers", "subject", "tone", "response_json"],  
    output_variables=["quiz", "review"],  
    verbose=True,
)


In [86]:
file_path = r"C:\Users\hp\genai\data.txt" 

In [87]:
with open(file_path, 'r') as file:
        TEXT = file.read()

In [88]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [89]:
NUMBERS = 5
SUBJECT = "data science"
TONE = "simple"

In [93]:
import json

In [99]:


# Assuming TEXT, NUMBERS, SUBJECT, TONE, and RESPONSE_JSON are defined
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "numbers": NUMBERS,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)  # Ensure RESPONSE_JSON is serializable
        }
    )

    print(response)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Abstract
The purpose was to specify the concept of traditional food and the dimensions that make it up, identifying: definitions, authors and research projects; as well as to determine what is known and possible topics for future research. A literature review of traditional foods was conducted that examined the conceptual development of the term. Social network analysis (SNA) was also used to identify the most relevant definitions and working groups on the topic. Twenty-three definitions were identified in the period 1995–2019. It reveals the difficulty of establishing one that encompasses such a dynamic concept. Although there is variability in the specific characteristics of these foods, four dimensions have been established: time, place, know-how, and cultural meaning. It was found that their main characteristic is the transmission of knowledge and raw materials between genera

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

NameError: name 'response' is not defined